In [3]:
import numpy as np
from scipy.fft import fft2, ifft2
from PIL import Image
import time

class GISTFeatureExtractor:
    def __init__(self, orientations=8, scales=4, blocks=4, boundary_extension=16):
        self.orientations = orientations
        self.scales = scales
        self.blocks = blocks
        self.boundary_extension = boundary_extension

    def create_gabor_filters(self, padded_image_size):
        filters = []
        fx, fy = np.meshgrid(
            np.linspace(-0.5, 0.5, padded_image_size[0]),
            np.linspace(-0.5, 0.5, padded_image_size[1]),
            indexing='ij'
        )

        fr = np.sqrt(fx**2 + fy**2)
        t = np.arctan2(fy, fx)

        for scale in range(self.scales):
            sigma = 0.35 * (0.8 ** scale)
            for orientation in range(self.orientations):
                theta = orientation * np.pi * 2 / self.orientations
                tr = t - theta
                tr = (tr + np.pi) % (2 * np.pi) - np.pi
                gabor = np.exp(
                    -10 * ((fr / sigma - 1)**2) - 2 * np.pi * (tr**2)
                )
                filters.append(gabor)
        return filters

    def preprocess_image(self, image_array):
        image_array = image_array - np.min(image_array)
        if np.max(image_array) > 0:
            image_array /= np.max(image_array)

        padded_image = np.pad(
            image_array,
            pad_width=self.boundary_extension,
            mode='reflect'
        )
        return padded_image

    def downsample_blocks(self, response, blocks):
        h, w = response.shape
        grid_h, grid_w = h // blocks, w // blocks
        downsampled = np.zeros((blocks, blocks))
        for i in range(blocks):
            for j in range(blocks):
                block = response[
                    i * grid_h:(i + 1) * grid_h,
                    j * grid_w:(j + 1) * grid_w
                ]
                downsampled[i, j] = block.mean()
        return downsampled

    def extract_gist_features(self, image):
        start_time = time.time()
        padded_image = self.preprocess_image(image)
        preprocess_time = time.time() - start_time

        padded_image_size = padded_image.shape
        filters = self.create_gabor_filters(padded_image_size)
        filter_time = time.time() - start_time - preprocess_time

        feature_time_start = time.time()
        features = []
        for gabor_filter in filters:
            response = np.abs(ifft2(fft2(padded_image) * fft2(gabor_filter)).real)
            response_blocks = self.downsample_blocks(response, self.blocks)
            features.append(response_blocks.flatten())
        gist_vector = np.concatenate(features)
        feature_extraction_time = time.time() - feature_time_start

        total_time = time.time() - start_time
        print(f"預處理時間: {preprocess_time:.4f} 秒")
        print(f"濾波器時間: {filter_time:.4f} 秒")
        print(f"特徵提取時間: {feature_extraction_time:.4f} 秒")
        print(f"總執行時間: {total_time:.4f} 秒")

        return gist_vector


orientations = 8
scales = 4
blocks = 4
boundary_extension = 16

image_path = "malware_image.png" 
image = Image.open(image_path).convert("L")
image_array = np.array(image, dtype=np.float32)

gist_extractor = GISTFeatureExtractor(orientations, scales, blocks, boundary_extension)
gist_features = gist_extractor.extract_gist_features(image_array)

print("GIST 特徵向量維度:", gist_features.shape)


預處理時間: 0.0020 秒
濾波器時間: 1.2625 秒
特徵提取時間: 2.0149 秒
總執行時間: 3.2795 秒
GIST 特徵向量維度: (512,)


In [ ]:
import torch
import numpy as np
from torch.fft import fft2, ifft2
from torchvision.transforms import functional as TF
from PIL import Image
import matplotlib.pyplot as plt
import time  # 計算執行時間

class GISTFeatureExtractor:
    def __init__(self, orientations=8, scales=4, blocks=4, boundary_extension=16, device="cuda"):
        self.orientations = orientations
        self.scales = scales
        self.blocks = blocks
        self.boundary_extension = boundary_extension
        self.device = device

    def create_gabor_filters(self, image_size):
        filters = []
        fx, fy = torch.meshgrid(
            torch.linspace(-0.5, 0.5, image_size[0], device=self.device),
            torch.linspace(-0.5, 0.5, image_size[1], device=self.device)
        )
        fr = torch.sqrt(fx**2 + fy**2)
        t = torch.atan2(fy, fx)

        for scale in range(self.scales):
            sigma = 0.35 * (0.8 ** scale)
            for orientation in range(self.orientations):
                theta = orientation * np.pi * 2 / self.orientations
                tr = t - theta
                tr = (tr + np.pi) % (2 * np.pi) - np.pi
                gabor = torch.exp(
                    -10 * ((fr / sigma - 1)**2) - 2 * np.pi * (tr**2)
                )
                filters.append(gabor)
        return filters

    def preprocess_image(self, image_tensor):
        image_tensor = image_tensor - image_tensor.min()
        if image_tensor.max() > 0:
            image_tensor /= image_tensor.max()

        if image_tensor.ndim == 2:
            image_tensor = image_tensor.unsqueeze(0).unsqueeze(0)

        image_tensor = torch.nn.functional.pad(
            image_tensor, (self.boundary_extension,) * 4, mode="reflect"
        )
        return image_tensor.squeeze(0).squeeze(0)

    def extract_gist_features(self, image):
        # 預處理圖像
        start_time = time.time()  # 開始計時
        image = self.preprocess_image(image)
        preprocess_time = time.time() - start_time

        image_size = image.shape[-2:]
        filters = self.create_gabor_filters(image_size)
        filter_time = time.time() - start_time - preprocess_time

        # 提取濾波響應並分塊
        feature_time_start = time.time()
        features = []
        for gabor_filter in filters:
            response = ifft2(fft2(image) * fft2(gabor_filter)).abs()
            response_blocks = self.downsample_blocks(response, self.blocks)
            features.append(response_blocks.flatten())
        gist_vector = torch.cat(features)
        feature_extraction_time = time.time() - feature_time_start

        total_time = time.time() - start_time
        print(f"預處理時間: {preprocess_time:.4f} 秒")
        print(f"濾波器生成時間: {filter_time:.4f} 秒")
        print(f"特徵提取時間: {feature_extraction_time:.4f} 秒")
        print(f"總執行時間: {total_time:.4f} 秒")

        return gist_vector

    def downsample_blocks(self, response, blocks):
        h, w = response.shape[-2:]
        grid_h, grid_w = h // blocks, w // blocks
        downsampled = torch.zeros((blocks, blocks), device=self.device)
        for i in range(blocks):
            for j in range(blocks):
                block = response[
                    i * grid_h:(i + 1) * grid_h, j * grid_w:(j + 1) * grid_w
                ]
                downsampled[i, j] = block.mean()
        return downsampled


orientations = 8
scales = 4
blocks = 4
boundary_extension = 16
device = "cuda" if torch.cuda.is_available() else "cpu"

# 載入灰階圖像
image_path = "malware_image.png"  # 替換為您的圖片路徑
image = Image.open(image_path).convert("L")
image_tensor = torch.tensor(np.array(image), dtype=torch.float32, device=device)

# 提取 GIST 特徵並計算執行時間
gist_extractor = GISTFeatureExtractor(orientations, scales, blocks, boundary_extension, device)
gist_features = gist_extractor.extract_gist_features(image_tensor)

print("GIST 特徵向量維度:", gist_features.shape)


預處理時間: 0.0020 秒
濾波器生成時間: 0.0031 秒
特徵提取時間: 0.1635 秒
總執行時間: 0.1685 秒
GIST 特徵向量維度: torch.Size([512])
